In [ ]:
# https://www.kaggle.com/dlarionov/feature-engineering-xgboost
import numpy as np
import pandas as pd

from itertools import product
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from math import sqrt

#import seaborn as sns
#import matplotlib.pyplot as plt
#%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from xgboost import XGBRegressor
from xgboost import plot_importance

from hyperopt import fmin, tpe, hp, STATUS_OK, STATUS_FAIL, Trials, space_eval

import time
import sys
import gc
import pickle

In [ ]:
def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

In [ ]:
data = pd.read_pickle('../data/data.pkl')
data = pd.read_pickle('../data/test.pkl')

In [ ]:
# Select features through trial and error
data = data[[
    'date_block_num',
    'shop_id',
    'item_id',
    'item_cnt_month',
    'city_code',
    'item_category_id',
    'type_code',
    'subtype_code',
    'item_cnt_month_lag_1',
    'item_cnt_month_lag_2',
    'item_cnt_month_lag_3',
    'item_cnt_month_lag_6',
    'item_cnt_month_lag_12',
    'date_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_2',
    'date_item_avg_item_cnt_lag_3',
    'date_item_avg_item_cnt_lag_6',
    'date_item_avg_item_cnt_lag_12',
    'date_shop_avg_item_cnt_lag_1',
    'date_shop_avg_item_cnt_lag_2',
    'date_shop_avg_item_cnt_lag_3',
    'date_shop_avg_item_cnt_lag_6',
    'date_shop_avg_item_cnt_lag_12',
    'date_cat_avg_item_cnt_lag_1',
    'date_shop_cat_avg_item_cnt_lag_1',
    'date_city_avg_item_cnt_lag_1',
    'date_item_city_avg_item_cnt_lag_1',
    'delta_price_lag',
    'month',
    'days',
    'item_shop_last_sale',
    'item_last_sale',
    'item_shop_first_sale',
    'item_first_sale',
]]

In [ ]:
# Validation strategy is 34 month for the test set,
#33 month for the validation set and 13-33 months for the train.
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']

X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']

X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [ ]:
del data
gc.collect();

In [ ]:
# Use XGBoost and hyperopt
# https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/
# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
# https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst#parameters-for-tree-booster
def score(params):
    model = XGBRegressor(**params)
    
    model.fit(X_train, Y_train, eval_set=[(X_train, Y_train), (X_valid, Y_valid)],
              verbose=False, early_stopping_rounds=10)
    Y_pred = model.predict(X_valid).clip(0, 20)
    score = sqrt(mean_squared_error(Y_valid, Y_pred))
    print(score)
    return {'loss': score, 'status': STATUS_OK}    
    
def optimize(trials):
    space = {
        'max_depth':hp.choice('max_depth', np.arange(10, 25, 1, dtype=int)),
        'n_estimators':hp.choice('n_estimators', np.arange(1000, 10000, 10, dtype=int)),
        'colsample_bytree':hp.quniform('colsample_bytree', 0.5, 1.0, 0.1),
        'min_child_weight':hp.choice('max_depth', np.arange(250, 350, 10, dtype=int)),
        'subsample':hp.quniform('colsample_bytree', 0.7, 0.9, 0.1),
        'eta':hp.quniform('eta', 0.1, 0.3, 0.1),
        
        'objective':'reg:squarederror',
        
        'tree_method':'gpu_hist',
        'eval_metric': 'rmse'
    }
    
    best = fmin(score, space, algo=tpe.suggest, max_evals=10)
    return best

In [ ]:
trials = Trials()
best_params = optimize(trials)

In [ ]:
# Return the best parameters
space = {
        'max_depth':hp.choice('max_depth', np.arange(10, 25, 1, dtype=int)),
        'n_estimators':hp.choice('n_estimators', np.arange(1000, 10000, 10, dtype=int)),
        'colsample_bytree':hp.quniform('colsample_bytree', 0.5, 1.0, 0.1),
        'min_child_weight':hp.choice('max_depth', np.arange(250, 350, 10, dtype=int)),
        'subsample':hp.quniform('colsample_bytree', 0.7, 0.9, 0.1),
        'eta':hp.quniform('eta', 0.1, 0.3, 0.1),
        
        'objective':'reg:squarederror',
        
        'tree_method':'gpu_hist',
        'eval_metric': 'rmse',
    }
space_eval(space, best_params)

In [ ]:
# Fit the model with the best parameters
model = XGBRegressor(
    max_depth=22,
    n_estimators=6830,
    colsample_bytree=0.5,
    min_child_weight=300,
    subsample=0.8,    
    eta=0.1,
    objective='reg:squarederror',
    tree_method='gpu_hist')

model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds=10)

In [ ]:
# Plot feature importance
plot_features(model, (10, 14))

In [ ]:
# Prediction
Y_pred = model.predict(X_valid).clip(0, 20)
Y_test = model.predict(X_test).clip(0, 20)

submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('../submission/xgb_submission.csv', index=False)

# save predictions for an ensemble
pickle.dump(Y_pred, open('../submission/xgb_train.pickle', 'wb'))
pickle.dump(Y_test, open('../submission/xgb_test.pickle', 'wb'))